# Iniciando projeto

In [1]:
import pandas as pd
import requests
from api_access import *

In [2]:
colunas = [
    'CMPLID', 'ODINO', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'CRASH', 'FAILDATE', 'FIRE', 
    'INJURED', 'DEATHS', 'COMPDESC', 'CITY', 'STATE', 'VIN', 'DATEA', 'LDATE', 'MILES', 'OCCURENCES', 
    'CDESCR', 'CMPL_TYPE', 'POLICE_RPT_YN', 'PURCH_DT', 'ORIG_OWNER_YN', 'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 
    'NUM_CYLS', 'DRIVE_TRAIN', 'FUEL_SYS', 'FUEL_TYPE', 'TRANS_TYPE', 'VEH_SPEED', 'DOT', 'TIRE_SIZE', 
    'LOC_OF_TIRE', 'TIRE_FAIL_TYPE', 'ORIG_EQUIP_YN', 'MANUF_DT', 'SEAT_TYPE', 'RESTRAINT_TYPE', 'DEALER_NAME', 
    'DEALER_TEL', 'DEALER_CITY', 'DEALER_STATE', 'DEALER_ZIP', 'PROD_TYPE', 'REPAIRED_YN', 'MEDICAL_ATTN', 
    'VEHICLES_TOWED_YN'
]


In [3]:
colunas_uteis_para_nlp = [
    "CMPLID",           # ID da Reclamação
    "MAKETXT",          # Marca do Veículo
    "MODELTXT",         # Modelo do Veículo
    "CRASH",            # Se envolvido em acidente
    "FAILDATE",         # Data do Incidente
    "CDESCR",           # Descrição da Reclamação (Campo principal para NLP)
    "CMPL_TYPE",        # Tipo da Reclamação (Ex: Defeito, Acidente, etc.)
    "POLICE_RPT_YN",    # Se foi reportado à polícia (Y/N)
    "VIN",              # Número de Identificação do Veículo (para referência)
    "STATE"             # Estado do Consumidor
]

In [4]:
# Realizar a requisição
response = requests.get(f'https://api.nhtsa.gov/complaints/complaintsByVehicle?make=acura&model=rdx&modelYear=2012')
lista = response.json()
# Verificar o status da resposta
if response.status_code == 200:
    dados = response.json()
    print(dados)
else:
    print(f"Erro ao obter dados: {response.status_code}")

{'count': 14, 'message': 'Results returned successfully', 'results': [{'odiNumber': 11132058, 'manufacturer': 'Honda (American Honda Motor Co.)', 'crash': False, 'fire': False, 'numberOfInjuries': 0, 'numberOfDeaths': 0, 'dateOfIncident': '08/01/2014', 'dateComplaintFiled': '09/28/2018', 'vin': '5J8TB2H27CA', 'components': 'EXTERIOR LIGHTING', 'summary': 'AMBOS FOCOS DELANTEROS SE OBEDECEN, LAS FECHAS EXACTAS CUANDO LO LLEV AL DEALER PARA GARANTA DE LOS FOCOS NO LA RECUERDO, PERO FUI EN VARIAS OCASIONES.  ME INFORMARON QUE LA ESPECIFICACIN DEL FABRICANTE ERAN FOCOS AL AIRE, Y QUE POR ESO SE HUMEDECIAN Y NO TENAN GARANTA, QUE TODOS LOS DE ESE AO VINIERON AS.  ME CONFIRM Y NO REGRES PARA ESE PROBLEMA., AUNQUE NO TIENE LGICA,  SE COMPRA UN AUTO PARA QUE FUNCIONE TODO NORMAL Y ESTE NO ES EL CASO DE AMBOS FOCOS.  AL PASAR EL TIEMPO EL PROBLEMA SE HA AGRAVADO PORQUE SE HUMEDECEN MAS Y AFECTA LA VISIBILIDAD.   LLAM NUEVAMENTE AL DEALER PERO ME INFORMARON LO MISMO, LES INFORM QUE FUI EN EL PER

In [5]:
data = lista['results']
df = pd.DataFrame(data)

In [6]:
df['products']

0     [{'type': 'Vehicle', 'productYear': '2012', 'p...
1     [{'type': 'Vehicle', 'productYear': '2012', 'p...
2     [{'type': 'Vehicle', 'productYear': '2012', 'p...
3     [{'type': 'Vehicle', 'productYear': '2012', 'p...
4     [{'type': 'Vehicle', 'productYear': '2012', 'p...
5     [{'type': 'Vehicle', 'productYear': '2012', 'p...
6     [{'type': 'Vehicle', 'productYear': '2012', 'p...
7     [{'type': 'Vehicle', 'productYear': '2012', 'p...
8     [{'type': 'Vehicle', 'productYear': '2012', 'p...
9     [{'type': 'Vehicle', 'productYear': '2012', 'p...
10    [{'type': 'Vehicle', 'productYear': '2012', 'p...
11    [{'type': 'Vehicle', 'productYear': '2012', 'p...
12    [{'type': 'Vehicle', 'productYear': '2012', 'p...
13    [{'type': 'Vehicle', 'productYear': '2012', 'p...
Name: products, dtype: object

In [7]:
# Função para extrair as informações de cada lista de dicionários
def expandir_products(row):
    if isinstance(row, list) and len(row) > 0:
        return row[0]  # Extraímos o primeiro dicionário da lista
    return {}

# Expandir a coluna 'products' e extrair as informações desejadas
df_expanded = pd.json_normalize(df['products'].apply(expandir_products))

# Concatenar os dados expandidos com o DataFrame original
df = pd.concat([df, df_expanded], axis=1)

# Exibir o DataFrame resultante
df

,odiNumber,manufacturer,crash,fire,numberOfInjuries,numberOfDeaths,dateOfIncident,dateComplaintFiled,vin,components,summary,products,type,productYear,productMake,productModel,manufacturer
0,11132058,Honda (American Honda Motor Co.),False,False,0,0,08/01/2014,09/28/2018,5J8TB2H27CA,EXTERIOR LIGHTING,"AMBOS FOCOS DELANTEROS SE OBEDECEN, LAS FECHAS...","[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
1,11124195,Honda (American Honda Motor Co.),False,False,0,0,06/16/2018,09/04/2018,5J8TB1H52CA,UNKNOWN OR OTHER,VEHICLE WILL COMPLETELY SHUT DOWN WHILE DRIVIN...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
2,11074082,Honda (American Honda Motor Co.),False,False,0,0,05/15/2016,02/21/2018,5J8TB2H2XCA,AIR BAGS,TAKATA RECALL. THE DRIVER AIR BAG WAS RECALLED...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
3,11035418,Honda (American Honda Motor Co.),False,False,0,0,10/20/2017,10/21/2017,5J8TB2H22CA,POWER TRAIN,I NOTICED VIBRATION WHILE DRIVING IN TOWN AS I...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
4,10897070,Honda (American Honda Motor Co.),False,False,0,0,02/01/2016,08/19/2016,5J8TB1H50CA,AIR BAGS,TL* THE CONTACT OWNS A 2012 ACURA RDX. THE CON...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
5,10894504,Honda (American Honda Motor Co.),False,False,0,0,02/03/2016,08/09/2016,5J8TB1H54CA,AIR BAGS,TL* TAKATA RECALL. THE CONTACT OWNS A 2012 ACU...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
6,10888250,Honda (American Honda Motor Co.),False,False,0,0,04/07/2016,07/25/2016,5J8TB1H59CA,AIR BAGS,TL* TAKATA RECALL. THE CONTACT OWNS A 2012 ACU...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
7,10877082,Honda (American Honda Motor Co.),False,False,0,0,03/02/2016,06/28/2016,5J8TB2H56CA,AIR BAGS,TL* TAKATA RECALL. THE CONTACT OWNS A 2012 ACU...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
8,10851890,Honda (American Honda Motor Co.),False,False,0,0,03/23/2016,03/28/2016,5J8TB1H23CA,AIR BAGS,TAKATA RECALL I RECEIVED A LETTER DATED MARCH...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)
9,10836547,Honda (American Honda Motor Co.),False,False,0,0,02/08/2016,02/16/2016,5J8TB2H20CA,"STEERING,POWER TRAIN",WITHIN THE FIRST 2 YEARS AN INDEPENDENT MECHAN...,"[{'type': 'Vehicle', 'productYear': '2012', 'p...",Vehicle,2012,ACURA,RDX,Honda (American Honda Motor Co.)


In [8]:
# get model year
data_year = get_makes_by_model_year(2023)

In [10]:
data_year['make'].unique()

array(['ACURA', 'AIRSTREAM', 'ALFA ROMEO', 'ALLIANCE RV', 'ALTEC',
       'AMERICAN COACH', 'AUDI', 'BAODIAO', 'BATTLE MOTORS', 'BENTLEY',
       'BIG TEX', 'BLUE BIRD', 'BMW', 'BUICK', 'CADILLAC', 'CAN-AM',
       'CFMOTO', 'CHEVROLET', 'CHRYSLER', 'COACHMEN', 'CRITERION',
       'DELCO TRAILERS', 'DODGE', 'DUCATI', 'DYNAMAX', 'ENERGICA',
       'ENTEGRA', 'FISKER', 'FLEETWOOD', 'FORD', 'FOREST RIVER',
       'FREIGHTLINER', 'GENESIS', 'GENESIS SUPREME', 'GMC',
       'GRAND DESIGN', 'HAPPIER CAMPER', 'HARLEY-DAVIDSON', 'HAUL-ABOUT',
       'HEARTLAND', 'HIGHLAND RIDGE', 'HOLIDAY RAMBLER', 'HONDA',
       'HUSQVARNA', 'HYUNDAI', 'IC BUS', 'INDIAN', 'INFINITI',
       'INTERNATIONAL', 'JAGUAR', 'JAYCO', 'JEEP', 'JUST TRAILERS',
       'KAUFMAN TRAILERS', 'KAWASAKI', 'KENWORTH', 'KEYSTONE', 'KIA',
       'KTM', 'LAMBORGHINI', 'LAND ROVER', 'LEXUS', 'LINCOLN', 'LUCID',
       'MACK', 'MARKET DIRECT CAMPERS', 'MASERATI', 'MAZDA', 'MCLAREN',
       'MERCEDES-BENZ', 'MERCEDES-MAYBACH', 'MIN